In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 02 11:51:55 2017

@author: John Lloyd

This module is used to interface with the TacTip family of optical tactile
sensors.

Some possible usage patterns are as follows:
    
1. Set up the interface, initialise the pin positions, and perform some basic
sensing operations:

import time
    
sensor = TactileSensorProxy(mask_type='circle',
                            mask_centre=(310, 230),
                            mask_radius=220)

pins = sensor.init_pins()
# sensor.set_pins(pins)

sensor.async_track_pins()  # non-blocking
time.sleep(5)
sensor.async_cancel()  # non-blocking
sensor.async_result()  # blocking

sensor.async_record_pins()  # non-blocking
time.sleep(1)
sensor.async_cancel()  # non-blocking
data = sensor.async_result()   # blocking

data = sensor.record_pins(10)  # blocking
"""

import sys
import logging

import cv2
import numpy as np
import scipy.spatial.distance as ssd

from datetime import datetime
from core.utils.process import SPCProxy

logger = logging.getLogger(__name__)


class TactileSensor:
    """
    TacTip and TacThumb interface class.
    """
    
    def __init__(self, *args, **kwargs):
        logger.debug('Entering TactileSensor.__init__')
<<<<<<< HEAD
        
<<<<<<< HEAD:python/tactile/sensor/tactile_sensor.py
        self._frame_width = frame_width
        self._frame_height = frame_height
        self._threshold = threshold
        self._min_pin_radius = min_pin_radius
        self._max_pin_radius = max_pin_radius
        self._max_tracking_move = max_tracking_move
        
        # Initialise mask for image processing
        if mask_type == 'circle':          
            centre = tuple(kwargs.get('mask_centre',
                          (np.int32(math.floor((self._frame_width + 200)/2)),
                           np.int32(math.floor((self._frame_height - 1)/2)))))           
            radius = kwargs.get('mask_radius', min(centre))
            self._mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
            cv2.circle(self._mask, centre, radius, 1, thickness=-1)                
        elif mask_type == 'rectangle':
            left = kwargs.get('mask_left', 0)
            top = kwargs.get('mask_top', 0)
            width = kwargs.get('mask_width', frame_width)
            height = kwargs.get('mask_height', frame_height)
            self._mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
            self._mask[top:top+height, left:left+width] = 255
        else:
            raise RuntimeError('unknown mask type') 
=======
        video_source = kwargs.get('video_source', 0)
        self._frame_width = kwargs.get('frame_width',640)        
        self._frame_height = kwargs.get('frame_height',480)
        brightness = kwargs.get('brightness',150)
        contrast = kwargs.get('contrast',10)
        saturation = kwargs.get('saturation',0)
        exposure = kwargs.get('exposure',-6)

        # Display camera image 
        self._camera_disp = kwargs.get('camera_disp=True', True)

        # Set blod detection parameters
        self._params = cv2.SimpleBlobDetector_Params()
        self._params.blobColor = 255
        # Change thresholds
        self._params.minThreshold = kwargs.get('minThreshold',5)
        self._params.maxThreshold = kwargs.get('maxThreshold',100)
        # Filter by Area.
        self._params.filterByArea = True
        self._params.minArea = kwargs.get('minArea',1000)
        self._params.maxArea = kwargs.get('maxArea',2000)
        # Filter by Circularity
        self._params.filterByCircularity = True        
        self._params.minCircularity = kwargs.get('minCircularity',0.3)
        # Filter by Convexity
        self._params.filterByConvexity = True
        self._params.minConvexity = kwargs.get('minConvexity',0.9)
        # Filter by Inertia
        self._params.filterByInertia = True        
        self._params.minInertiaRatio = kwargs.get('minInertiaRatio',0.2)         
>>>>>>> master:python/core/sensor/tactile_sensor.py
        
        self._pin_tracking = kwargs.get('pin_tracking', True) # Initialise tracking 
        self._max_tracking_move = kwargs.get('max_tracking_move', 20)  
        max_pin_dist_from_centroid = kwargs.get('max_pin_dist_from_centroid', 300)
        min_pin_separation = kwargs.get('min_pin_separation',0)

        self._camera_channel = kwargs.get('camera_channel', 3) # Initialise Camera Channel # 0 - blue, 1 - green, 2 - red, 3 - grey (and defalt)

        # Initialise webcam          
        max_attempts = 5
=======

        # Initialise camera and set properties
        video_source = kwargs.get('video_source', 0)
        MAX_ATTEMPTS = 5
>>>>>>> f96880b9b43962928d3f16ed7582e973bbba49b4
        self._cap = cv2.VideoCapture(video_source)
        attempts = 1
        while not self._cap.isOpened() and attempts < MAX_ATTEMPTS:
            self._cap = cv2.VideoCapture(video_source)
            attempts += 1
        if not self._cap.isOpened():
            raise RuntimeError('failed to initialise video source')
        N_INIT_FRAMES_DUMP = 3
        for i in range(N_INIT_FRAMES_DUMP):
            ret, frame = self._cap.read()
            
        self._frame_width = kwargs.get('frame_width', 640)
        self._cap.set(cv2.CAP_PROP_FRAME_WIDTH, self._frame_width)
        self._frame_height = kwargs.get('frame_height', 480)
        self._cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self._frame_height)        
        brightness = kwargs.get('brightness', 150)
        self._cap.set(cv2.CAP_PROP_BRIGHTNESS, brightness)
        contrast = kwargs.get('contrast', 10)
        self._cap.set(cv2.CAP_PROP_CONTRAST, contrast)
        saturation = kwargs.get('saturation', 0)
        self._cap.set(cv2.CAP_PROP_SATURATION, saturation)
        exposure = kwargs.get('exposure', -6)
        self._cap.set(cv2.CAP_PROP_EXPOSURE, exposure)

        # Initialise other camera-related parameters        
        self._camera_disp = kwargs.get('camera_disp=True', True)        
        self._camera_channel = kwargs.get('camera_channel', 3)

        # Initialise window for displaying camera images
        cv2.namedWindow('sensor', cv2.WINDOW_NORMAL)

        # Initialise blob detector
        self._params = cv2.SimpleBlobDetector_Params()
        self._params.blobColor = 255
        self._params.minThreshold = kwargs.get('min_threshold', 50)
        self._params.maxThreshold = kwargs.get('max_threshold', 150)
        self._params.filterByArea = True
        self._params.minArea = kwargs.get('min_area', 40)
        self._params.maxArea = kwargs.get('max_area', 120)
        self._params.filterByCircularity = True        
        self._params.minCircularity = kwargs.get('min_circularity', 0.3)
        self._params.filterByConvexity = True
        self._params.minConvexity = kwargs.get('min_convexity', 0.3)
        self._params.filterByInertia = True        
        self._params.minInertiaRatio = kwargs.get('min_inertia_ratio', 0.1)
        self._detector = cv2.SimpleBlobDetector_create(self._params)

        # Initialise pin positions  
        self._pin_tracking = kwargs.get('pin_tracking', True)
        if self._pin_tracking:
            self._max_tracking_move = kwargs.get('max_tracking_move', 20)
            max_pin_dist_from_centroid = kwargs.get('max_pin_dist_from_centroid', 300)
            min_pin_separation = kwargs.get('min_pin_separation', 0)
            try:
                self.init_pins(max_pin_dist_from_centroid, min_pin_separation) 
            except:
                self._cap.release()
                cv2.destroyAllWindows()
                raise
        
        logger.debug('Leaving TactileSensor.__init__')        

    def __del__(self):
        logger.debug('Entering TactileSensor.__del__')
        if self._cap.isOpened():
            self._cap.release()
        cv2.destroyAllWindows()
        logger.debug('Leaving TactileSensor.__del__') 

    def _select_pins(self, pins, max_dist_from_centroid, min_separation):
        logger.debug('Entering TactileSensor._select_pins')
        # Calcualte centroid of pin positions 
        centroid = np.mean(pins, axis=0)
        centroid = centroid[np.newaxis,:]
        # Calculate distances of pins from centroid
        pin_rads = np.squeeze(ssd.cdist(pins, centroid, 'euclidean'))
        # Filter out pins that are further than specified distance from centroid
        pins = pins[pin_rads <= max_dist_from_centroid,:]
        
        # Calculate distances between each pair of pins
        pin_dists = ssd.cdist(pins, pins, 'euclidean')
        processed = np.zeros(pins.shape[0], dtype=bool)
        selected = np.zeros(pins.shape[0], dtype=bool)
        # Add pin closest to centroid to selected pins
        idx = np.argmin(ssd.cdist(pins, centroid, 'euclidean'))
        selected[idx] = True
        while not np.all(processed):
            # Mark selected pins and pins closer than minimum separation as
            # processed
            processed = processed | selected | (pin_dists[idx,:] < min_separation)     
            pin_dists[idx, processed] = np.inf
            # Add closest unprocessed pin to selected pins
            idx = np.argmin(pin_dists[idx,:])
            selected[idx] = True
        pins = pins[selected,:]
        
        # Sort pins by ascending x coordinate
        pins = pins[pins[:,0].argsort(),:]
        logger.debug('Leaving TactileSensor._select_pins')
        return pins

    def _map_pins(self, pins, prev_pins):
        logger.debug('Entering TactileSensor._map_pins')
        # Map pin positions to closest matching previous position
        pin_dists = ssd.cdist(pins, prev_pins, 'euclidean')
        min_pin_idxs = np.argmin(pin_dists, axis=0)
        pins = pins[min_pin_idxs,:]
        # Overwrite with previous pin position if pin move exceeds threshold
        min_pin_dists = np.min(pin_dists, axis=0)        
        rep_pin_idxs = (min_pin_dists > self._max_tracking_move)
        pins[rep_pin_idxs,:] = prev_pins[rep_pin_idxs,:]
        logger.debug('Leaving TactileSensor._map_pins')
        return pins

<<<<<<< HEAD
<<<<<<< HEAD:python/tactile/sensor/tactile_sensor.py
    def _draw_pins(self, frame, pins):
        logger.debug('Entering TactileSensor._draw_pins')        
        for p in pins:
            cv2.circle(frame, tuple(np.int32(p)), 3, (0,0,255), -1)
        frame = cv2.bitwise_and(frame, frame, mask=self._mask)
<<<<<<< HEAD

=======
>>>>>>> 2384982e557acfc748ec0e136042acc53ef45c0f
        logger.debug('Leaving TactileSensor._draw_pins') 
        return frame
=======
    #def _draw_pins(self, frame, pins):
        #logger.debug('Entering TactileSensor._draw_pins')
        #for p in pins:
        #    cv2.circle(frame, tuple(np.int32(p)), 3, (0,0,255), -1)
        #frame = cv2.bitwise_and(frame, frame, mask=self._mask)
        #logger.debug('Leaving TactileSensor._draw_pins') 
        #return frame
>>>>>>> master:python/core/sensor/tactile_sensor.py
=======
    def close(self):
        logger.debug('Entering TactileSensor.close') 
        self._cap.release()
        cv2.destroyAllWindows()
        logger.debug('Leaving TactileSensor.close') 

    def detect_pins(self, frame):
        logger.debug('Entering TactileSensor.detect_pins')
        # Capture frame-by-frame
        if frame is None:
            ret, frame = self._cap.read()
        # Our operations on the frame come here
        if self._camera_channel == 3:
            # Convert sensor frame to grayscale 
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        else:        
            # Convert sensor frame to b,g, or r 
            bgr = cv2.split(frame)
            frame = bgr[self._camera_channel]

        # Get keypoints and pin positions
        keypoints = self._detector.detect(frame)
        pins = np.array([k.pt for k in keypoints])
        
        logger.debug('Leaving TactileSensor.detect_pins')
        return pins, keypoints
>>>>>>> f96880b9b43962928d3f16ed7582e973bbba49b4
    
    def init_pins(self, max_pin_dist_from_centroid=300, min_pin_separation=0,
                  filename=None):
        logger.debug('Entering TactileSensor.init_pins')
        if filename is not None:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(filename, fourcc, 20.0,
                                  (self._frame_width,self._frame_height))
        ret, frame = self._cap.read()
        if ret:
            if filename is not None: out.write(frame)
            pins, keypoints = self.detect_pins(frame)
            im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            if self._camera_disp == True:
                cv2.imshow('sensor', im_with_keypoints)
                cv2.waitKey(1)
            if pins.shape[0] == 0:
                raise RuntimeError('failed to identify any pins in frame')
            self._pins = self._select_pins(pins, max_pin_dist_from_centroid,
                                           min_pin_separation) 
        else:
            raise RuntimeError('failed to capture initialisation frame')
        if filename is not None: out.release()
        logger.debug('Leaving TactileSensor.init_pins')
        return self._pins

    def set_pins(self, pins):
        logger.debug('Entering TactileSensor.set_pins')
        self._pins = pins
        logger.debug('Leaving TactileSensor.set_pins')
        
    def track_pins(self, filename=None, cancel=None):
        logger.debug('Entering TactileSensor.track_pins')
        
        if not self._pin_tracking:
            raise RuntimeError('can\'t track pins with pin tracking disabled')
        
        if filename is not None:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(filename, fourcc, 20.0,
                                  (self._frame_width,self._frame_height))
        while True:
            if cancel is not None and cancel(): break
            ret, frame = self._cap.read()
            if ret:
                if filename is not None: out.write(frame)
                pins, keypoints = self.detect_pins(frame)
                if pins.shape[0] == 0:
                    raise RuntimeError('failed to identify any pins in frame')
                self._pins = self._map_pins(pins, self._pins)
                im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                if self._camera_disp == True:
                    cv2.imshow('sensor', im_with_keypoints)
                    cv2.waitKey(1)
            else:
                raise RuntimeError('failed to capture frame')
        if filename is not None: out.release()
        logger.debug('Leaving TactileSensor.track_pins')

    def record_pins(self, num_samples=1000000, filename=None, cancel=None):
        logger.debug('Entering TactileSensor.record_pins')
        if filename is not None:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(filename, fourcc, 20.0,
                                  (self._frame_width,self._frame_height))            
        pins_list = []
        for _ in range(num_samples):
            if cancel is not None and cancel(): break            
            ret, frame = self._cap.read()
            if ret:      
                if filename is not None: out.write(frame)
                pins, keypoints = self.detect_pins(frame)
                im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                if self._camera_disp == True:
                    cv2.imshow('sensor', im_with_keypoints)
                    cv2.waitKey(1)
                if pins.shape[0] == 0:
                    raise RuntimeError('failed to identify any pins in frame')
                if self._pin_tracking:
                    self._pins = self._map_pins(pins, self._pins)
                else:
                    self._pins = pins
                pins_list.append(self._pins)
            else:
                raise RuntimeError('failed to capture frame')  
        pins_list = np.array(pins_list)
        if filename is not None: out.release()
        
        logger.debug('Leaving TactileSensor.record_pins')        
        return pins_list

    def record_pins_with_time_stamp(self, num_samples=1000000, filename=None, cancel=None):
        logger.debug('Entering TactileSensor.record_pins_with_time_stamp')
        if filename is not None:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(filename, fourcc, 20.0,
                                  (self._frame_width,self._frame_height))            
        pins_list = []
        time_stamp_list = []
        for _ in range(num_samples):
            if cancel is not None and cancel(): break            
            ret, frame = self._cap.read()
            if ret:
                time_stamp_list.append(str(datetime.now()))
                if filename is not None: out.write(frame)
                pins, keypoints = self.detect_pins(frame)
                im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                if self._camera_disp == True:
                    cv2.imshow('sensor', im_with_keypoints)
                    cv2.waitKey(1)
                if pins.shape[0] == 0:
                    raise RuntimeError('failed to identify any pins in frame')
                self._pins = self._map_pins(pins, self._pins)
                pins_list.append(self._pins)
            else:
                raise RuntimeError('failed to capture frame')  
        pins_list = np.array(pins_list)
        if filename is not None: out.release()
        
        logger.debug('Leaving TactileSensor.record_pins_with_time_stamp')        
        return (pins_list, time_stamp_list)        

    def record_frames(self, num_samples=1000000, filename=None, cancel=None):
        logger.debug('Entering TactileSensor.record_frames')
        if filename is not None:
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')
            out = cv2.VideoWriter(filename, fourcc, 20.0,
                                  (self._frame_width,self._frame_height))            
        frames = []
        for _ in range(num_samples):
            if cancel is not None and cancel(): break            
            ret, frame = self._cap.read()
            if ret:      
                if filename is not None: out.write(frame)
                if self._camera_disp == True:
                    cv2.imshow('sensor', frame)
                    cv2.waitKey(1)
                frames.append(frame)
            else:
                raise RuntimeError('failed to capture frame')  
        frames = np.array(frames)
        if filename is not None: out.release()
        
        logger.debug('Leaving TactileSensor.record_frames')        
        return frames


class TactileSensorProxy(SPCProxy):
    """Proxy class for interfacing with a TactileSensor object running in a
    separate process. All synchronous and asynchronous method calls are handled
    by the SPCProxy base class.
    """
    def __init__(self, *args, **kwargs):
        SPCProxy.__init__(self, TactileSensor, *args, **kwargs)
    

def main():
#    sensor = TactileSensorProxy(pin_tracking=False)
#    try:
#        frames = sensor.record_frames(100)
#        print(frames.shape)
#    finally:
#        del sensor
    
    sensor = TactileSensorProxy()
    try:
        pins = sensor.record_pins(100)
        print(pins.shape)
    finally:
        del sensor    
  
                      
if __name__ == '__main__':
    logging.basicConfig(stream=sys.stderr)
    logging.getLogger(__name__).setLevel(logging.DEBUG)
    main()


SyntaxError: invalid syntax (<ipython-input-2-11919fbd7216>, line 57)